# Create type and streams


Objective load a dataset and run a pre-defined set of analyses

This set of notebooks is based upon 
1. NASA bearing dataset: http://data-acoustics.com/measurements/bearing-faults/bearing-4/
   Set number 2: 4 accelerometers one on each bearing
2. Tutorial: https://towardsdatascience.com/machine-learning-for-anomaly-detection-and-condition-monitoring-d4614e7de770

Acknowledgement is made for the measurements used in this work provided through data-acoustics.com Database

Requirements:

1. Juptyer notebook or lab
2. Python 
3. ocs_sample_library https://github.com/osisoft/OSI-Samples/tree/master/ocs_samples/library_samples/Python3
4. config.ini configured for your OSIsoft Cloud Services environment. Example here:
https://github.com/osisoft/OSI-Samples/tree/master/ocs_samples/basic_samples/SDS/Python/SDSPy/Python3


# Note: Follow the instructions in the notebook '0. configure...' prior to running this notebook

In [1]:
import configparser
import logging
from ocs_sample_library_preview import *
import json

config = configparser.ConfigParser()
config.read('config.ini')

ocsClient = OCSClient(config.get('Access', 'ApiVersion'), config.get('Access', 'Tenant'), config.get('Access', 'Resource'), 
                        config.get('Credentials', 'ClientId'), config.get('Credentials', 'ClientSecret'))
        

namespace_id = config.get('Configurations', 'Namespace')
print(namespace_id)

RS-MEA


# Create a type and streams to describe the bearing data

In [2]:
# Dictionary to describe information about the type
thetype = {'sdstypecode': SdsTypeCode.Object, 
           'id': 'bearing',
           'name': 'bearing', 
           'description': 'bearing vibration signals'}

# sds type properties
timestamp_property = SdsTypeProperty()
timestamp_property.Id = "timestamp"
timestamp_property.SdsType = SdsType.fromDictionary({"SdsTypeCode": SdsTypeCode.DateTime.value})
timestamp_property.IsKey = True

index_property = SdsTypeProperty()
index_property.Id = "index"
index_property.SdsType = SdsType.fromDictionary({"SdsTypeCode": SdsTypeCode.Decimal.value})

channel_property =  SdsTypeProperty()
channel_property.Id = "channel"
channel_property.SdsType = SdsType.fromJson({"SdsTypeCode" : SdsTypeCode.Double.value})

# sds type defintion
bearing = SdsType()
bearing.Id = thetype['id']
bearing.SdsTypeCode = thetype['sdstypecode']
bearing.Name = thetype['name']
bearing.Description=thetype['description']
bearing.Properties = [timestamp_property,index_property,channel_property]

bearing_type = ocsClient.Types.getOrCreateType(namespace_id, bearing)
#print(json.dumps(json.loads(bearing_type.toJson()), indent=4))

In [3]:
# create the streams using the defined type
for bearing in range (1,5):
    stream = SdsStream(id=f'NASA.Bearing{bearing}',name=f'bearing{bearing}', description=f'Bearing {bearing} Channel 1', typeId='bearing')
    stream = ocsClient.Streams.getOrCreateStream(namespace_id,stream)
    print(f'id: {stream.Id}, name: {stream.Name}')

id: NASA.Bearing1, name: bearing1
id: NASA.Bearing2, name: bearing2
id: NASA.Bearing3, name: bearing3
id: NASA.Bearing4, name: bearing4


# Create a type and streams to store the aggregate value for each sample

In [4]:
# Dictionary to describe information about the type
thetype = {'sdstypecode': SdsTypeCode.Object, 
           'id': 'bearing.agg',
           'name': 'bearing channel aggregate', 
           'description': 'bearing vibration signals'}

# sds type defintion
bearing = SdsType()
bearing.Id = thetype['id']
bearing.SdsTypeCode = thetype['sdstypecode']
bearing.Name = thetype['name']
bearing.Description=thetype['description']
# use previous definitions for properties
bearing.Properties = [timestamp_property,channel_property]

bearing_type = ocsClient.Types.getOrCreateType(namespace_id, bearing)
#print(json.dumps(json.loads(bearing_type.toJson()), indent=4))

In [10]:
# create the streams using the defined type
for bearing in range (1,5):
    stream = SdsStream(id=f'NASA.Bearing{bearing}.agg',name=f'bearing{bearing} aggregate', description=f'Bearing {bearing} Channel 1', typeId='bearing.agg')
    ocsClient.Streams.createOrUpdateStream(namespace_id,stream)
    print(f'id: {stream.Id}, name: {stream.Name}')

id: NASA.Bearing1.agg, name: bearing1 aggregate
id: NASA.Bearing2.agg, name: bearing2 aggregate
id: NASA.Bearing3.agg, name: bearing3 aggregate
id: NASA.Bearing4.agg, name: bearing4 aggregate
